<a href="https://colab.research.google.com/github/yeonjin99/TIL/blob/main/LangChain_Expression_Language(LCEL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain 라이브러리를 사용하여 다양한 체인을 만들어 자연어 처리 및 정보 검색 작업 수행

## 1.0 Install the required packages

In [ ]:
!pip install langchain langchain-openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your_api_key"

### 1.0.1 LLM 호출
- 현 예제에서는 Docker가 무엇인지, 배포에서의 유용성이 무엇인지에 대해 묻습니다.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

llm.invoke("What is docker and how is it useful for deployment?")

## 2.0 Create your first chain with LCEL
- 사용자의 입력을 받아서 영어 문장을 프랑스어로 번역하는 체인 생성
- 체인 호출 결과는 프롬프트에 기반해 번역된 문장을 반환

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an English-French translator that return whatever the user says in French"),
    ("user", "{input}")
])

chain = prompt | llm

chain.invoke({
    "input" : "i enjoy going to rock concerts"
})

In [ ]:
# add output parser to the chain

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "my friend robert has a blue cat"})

In [ ]:
llm.invoke("What is new in langchain 0.1.0?")

## 3.0 Create a Retrieval Chain

### 3.1 Load the source documents and build vector store

In [ ]:
# retrieval chain

!pip install beautifulsoup4
!pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langchain-v0-1-0/")

docs = loader.load()

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter  import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [ ]:
documents

In [ ]:
vectorstore = FAISS.from_documents(documents, embeddings)

### 3.2 Create a Context-Aware LLM Chain

- ```create_stuff_documents_chain```을 사용해 문서 기반 질문 응답 체인 생성. 여기서는 ```LangChain 0.1.0에 대한 정보를 기반으로 질문에 답변
- ```context``` 부분에 문서 내용을 넣어 질문에 대한 답변 생성

In [ ]:
# create chain for documents

from langchain.chains.combine_documents import create_stuff_documents_chain

template = """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "what is langchain 0.1.0?",
    "context": [Document(page_content="langchain 0.1.0 is the new version of a llm app development framework.")]
})

### 3.2 Create the RAG Chain

In [ ]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({
    "input": "what is new in langchain 0.1.0"
})

response["answer"]

## 4.0 Create Conversational RAG Chain

### 4.1 Create a Conversation-Aware Retrieval Chain

- 과거 대화 내용을 반영한 검색 체인 생성. 여기서는 대화 기록을 기반으로 추가적인 검색 쿼리를 생성

In [ ]:
# conversational retrieval chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

## 4.2 Use Retrieval Chain together with Document Chain

In [ ]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "What is langchain 0.1.0 about?"
})

In [ ]:
response

In [ ]:
response["answer"]

In [ ]:
# simulate conversation history

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history' : chat_history,
    "input": "Tell me more about it!"
})

In [ ]:
response

In [ ]:
response["answer"]